In [ ]:
%pip install kagglehub

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ahmedwaelnasef/cars-dataset")
!cp {path}/cars.csv data/cars.csv


In [37]:
import pandas as pd
import numpy as np
import re

df[df['price'] == 'Grigio Maratea']

,car name,price,engine_capacity,cylinder,horse_power,top_speed,seats,brand,country
845,Maserati Levante 2021 3.0T V6 S,Grigio Maratea,3.0,6,430,264,5 Seater,maserati,ksa
2107,Maserati Levante 2021 3.0T V6 S,Grigio Maratea,3.0,6,430,264,5 Seater,maserati,bahrain
2117,Maserati Levante 2021 3.8T V8 GTS,Grigio Maratea,3.8,8,550,292,5 Seater,maserati,bahrain
3043,Maserati Levante 2021 3.0T V6 GranSport,Grigio Maratea,3.0,6,350,251,5 Seater,maserati,qatar
3065,Maserati Levante 2021 3.0T V6 S GranLusso,Grigio Maratea,3.0,6,430,264,5 Seater,maserati,qatar
3067,Maserati Levante 2021 3.0T V6 S,Grigio Maratea,3.0,6,430,264,5 Seater,maserati,qatar
3073,Maserati Levante 2021 3.8T V8 GTS,Grigio Maratea,3.8,8,550,292,5 Seater,maserati,qatar
3083,Maserati Levante 2021 3.0T V6 S GranSport,Grigio Maratea,3.0,6,430,264,5 Seater,maserati,qatar
3946,Maserati Levante 2021 3.0T V6 GranSport,Grigio Maratea,3.0,6,350,251,5 Seater,maserati,oman
3968,Maserati Levante 2021 3.8T V8 GTS,Grigio Maratea,3.8,8,550,292,5 Seater,maserati,oman


In [39]:
df = pd.read_csv('data/cars.csv')
df.head()
df['seats'].value_counts()  

filtered = df['car name' == 'Maserati Levante 2021 3.0T V6 S']

filtered['price'].str.contains(r'(\d+)')

KeyError: False

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6308 entries, 0 to 6307
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   car name         6308 non-null   object
 1   price            6308 non-null   object
 2   engine_capacity  6308 non-null   object
 3   cylinder         5684 non-null   object
 4   horse_power      6308 non-null   object
 5   top_speed        6308 non-null   object
 6   seats            6308 non-null   object
 7   brand            6308 non-null   object
 8   country          6308 non-null   object
dtypes: object(9)
memory usage: 443.7+ KB


In [14]:
df.isna().sum()

car name             0
price                0
engine_capacity      0
cylinder           624
horse_power          0
top_speed            0
seats                0
brand                0
country              0
dtype: int64

In [16]:
df['cylinder'].value_counts()

cylinder
4                2856
6                1480
8                 924
3                 139
12                136
N/A, Electric     107
10                 21
5                  17
Drive Type          3
16                  1
Name: count, dtype: int64

In [17]:
df['country'].value_counts()

country
uae        1248
ksa         996
kuwait      932
qatar       925
oman        910
bahrain     906
egypt       391
Name: count, dtype: int64

In [18]:
df['brand'].value_counts()

brand
mercedes-benz    560
audi             398
bmw              394
toyota           378
ford             323
                ... 
tata               2
soueast            2
byd                2
dfm                1
bugatti            1
Name: count, Length: 82, dtype: int64

In [26]:
df['cylinder'] = df['cylinder'].replace('N/A, Electric', pd.NA)
df.isna().sum()

car name             0
price                0
engine_capacity      0
cylinder           731
horse_power          0
top_speed            0
seats                0
brand                0
country              0
dtype: int64

In [27]:
df.dropna(inplace=True)


In [29]:
df['price'].value_counts()

price
TBD                     365
Following               217
DISCONTINUED            134
Grigio Maratea           23
Satin Steel Metallic     17
                       ... 
BHD 22,800                1
BHD 23,300                1
BHD 23,400                1
BHD 27,356                1
AED 1,766,100             1
Name: count, Length: 3109, dtype: int64

In [6]:
df['top_speed'].value_counts()
df['seats'].value_counts()

seats
5 Seater    3471
4 Seater     847
7 Seater     532
2 Seater     428
8 Seater     211
            ... 
24.1           1
12.3           1
230            1
220            1
2.8            1
Name: count, Length: 82, dtype: int64

In [21]:
 

df['seats'] = df['seats'].str.extract('(\d+)')

df['seats'].value_counts()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_3423/4180471034.py:1: SyntaxWarning: invalid escape sequence '\d'
  df['seats'] = df['seats'].str.extract('(\d+)')


seats
5      3488
4       875
7       586
2       438
8       214
6       162
3       124
9       110
11       27
15       25
10       22
200      20
260      16
12       15
150      12
17       11
14        7
185       6
190       6
160       5
80        4
18        4
30        3
240       2
250       2
156       2
120       1
145       1
13        1
230       1
220       1
24        1
Name: count, dtype: int64

'8'